In [ ]:
:dep num

In [ ]:
println!("Hello");

Hello


In [ ]:
:dep itertools
:dep statrs

In [ ]:
use statrs::distribution::{LogNormal, Poisson, Normal, Continuous, Discrete};
use statrs::function::logistic::{logit, logistic};


In [ ]:
let v = logit(0.0);
v

-inf

In [ ]:
let v = logit(-0.0);
v

-inf

In [ ]:
let x = logistic(0.0);
x

0.5

In [ ]:
:dep showata = { version = "0.3", features=["show_ndarray"]}
:dep ndarray = "0.14"

use showata::Showable;
use ndarray::Array2;


let data: Array2<f64> = Array2::zeros((3, 4));
data.show()


,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [ ]:
:dep itertools
:dep statrs
:dep assert_approx_eq
:dep anyhow

use itertools::izip
use statrs::distribution::{LogNormal, Poisson, Normal, Continuous, Discrete};
use statrs::function::logistic::{logit, logistic, checked_logit};
use assert_approx_eq::assert_approx_eq
use anyhow::Result;


In [ ]:
// Transform a variable defined over [0,1] to a variable defined over [-inf, +inf]
fn transform_constrained_to_unconstrained(x:f64) -> f64{
    const a: f64 = 0.0;
    const b: f64 = 1.0;
    let y = 
        if x==a {
            -f64::INFINITY
        }
        else if x == b {
            f64::INFINITY 
        }
        else {
            let p = (x-a)/(b-a);
            let result = checked_logit( p );
            match result{
                Ok(y) => y,
                // TODO: attach context to the error using ThisError or Anyhow
                Err(e) => { println!("p={}",p); panic!("{}",e);},
            }
        };
    y
}

// Transform a variable defined over (-inf,inf) to a variable defined over (0,1)
fn transform_unconstrained_to_constrained(y:f64) -> f64{
    const a: f64 = 0.0;
    const b: f64 = 1.0;
    let x = a + (b-a) * logistic(y);
    x
}



In [ ]:
fn is_power_of_two(num:usize) -> bool
{
    num & (num - 1) == 0
}

fn log2_usize(num:usize) -> usize
{
    let mut tmp = num;
    let mut shift_count = 0;
    while tmp > 0 {
        tmp = tmp >> 1;
        shift_count += 1;
    }
    shift_count - 1
}

// Transform radon concentrations from actual values into a simpler-to-sample form
fn transform_radon_concs(radon_conc: &mut[f64]) -> Result<()>{
    let n = radon_conc.len();
    assert!(is_power_of_two(n));
    let num_levels = log2_usize(n);
    let mut row = radon_conc.to_owned();
    let mut params: Vec<f64> = Vec::new();
    for _ in 0..num_levels{
        // pair elements, and then take average of consecutive elements
        params.extend(row.chunks_exact(2).map(|w| w[0] / (w[0]+w[1]/2.0)));
        row = row.chunks_exact(2).map(|w| w[0]+w[1]/2.0).collect();
        //row.clear();
        //row.extend(tmp.iter());
        //println!("{:?}", row);

    }
    assert!(row.len() == 1);
    params.extend(row);
    assert!(radon_conc.len() == params.len());
    for ii in 0..n{
        radon_conc[ii] = transform_unconstrained_to_constrained(params[ii]/2.0);
    }
    

    Ok(())
}

// Reverse transform radon concentration (from sampling form back to true values)
fn inverse_transform_radon_concs(p: &mut[f64]) -> Result<()>{
    let npts = p.len();
    assert!(is_power_of_two(npts));
    let num_levels = log2_usize(npts);

    let mut params = p.iter().map(|itm| transform_constrained_to_unconstrained(*itm)).collect::<Vec<_>>();
    let mut n = 1;
    let mut a: Vec<f64> = vec![params.pop().unwrap()];
    
    let rp = &params[..];
    
    for _ in 1..num_levels+1{
        // parameters for this level of the reconstruction
        let p = &rp[rp.len()-n..rp.len()];
        // remaining parameters
        let rp = &[..rp.len()-n];
        // reconstruct this level
        a = izip!(a,p).map(|ap| {let (a,p) = ap; [a*p, a*(2.0-p)]}).flatten().collect();
        n *= 2
    }
    for ii in 0..npts{
        p[ii] = a[ii];
    }


    Ok(())
}


In [ ]:
transform_constrained_to_unconstrained(0.5)

0.0

In [ ]:
let npts = 64;
let radon: Vec<f64> = (0..npts).map(|itm| itm as f64).collect();
radon

,0
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0
5,5.0
6,6.0
7,7.0
8,8.0
9,9.0


In [ ]:
let mut radon_transformed = radon.clone();
transform_radon_concs(& mut radon_transformed).unwrap();
radon_transformed

,0
0,0.5
1,0.5709465968834663
2,0.5763218799074524
3,0.5782977732960317
4,0.5793242521487495
5,0.5799530460821379
6,0.5803777601039839
7,0.5806838754763852
8,0.580914982687901
9,0.5810956417872526


In [ ]:
let mut radon_reconstructed = radon_transformed.clone();
inverse_transform_radon_concs(& mut radon_reconstructed).unwrap();
radon_reconstructed

,0
0,inf
1,inf
2,inf
3,inf
4,inf
5,inf
6,inf
7,inf
8,inf
9,inf


In [ ]:
let p = radon_transformed.clone();
let npts = p.len();
let num_levels = log2_usize(npts);
(npts,num_levels)

(64, 6)

In [ ]:
let mut params = p.iter().map(|itm| 2.0*transform_constrained_to_unconstrained(*itm)).collect::<Vec<_>>();
params

,0
0,0.0
1,0.5714285714285714
2,0.6153846153846154
3,0.631578947368421
4,0.6400000000000005
5,0.6451612903225806
6,0.6486486486486484
7,0.6511627906976737
8,0.653061224489796
9,0.654545454545455


In [ ]:
transform_unconstrained_to_constrained(0.0)

0.5

In [ ]:
use num::Float

pub fn exp_transform<P: Float>(u: P) -> (P, P) {
    // | d/du (exp(u)) | = exp(u)
    // log(|exp(u)|) = u
    let mut sigma = u.exp();
    if sigma > P::max_value(){
        sigma = P::max_value();
    }
    if sigma < P::max_value(){
        sigma = P::min_value();
    }
    
    (sigma, u)
}


In [ ]:
exp_transform(-10.0)

(-1.7976931348623157e308, -10.0)

In [ ]:
exp_transform(1e99)

(1.7976931348623157e308, 1e99)

In [ ]:
(f64::MAX).ln()

709.782712893384

In [ ]:
f64::MIN.exp()

0.0

In [ ]:
(10.0).exp()

22026.465794806718

In [ ]:
(0.06253275212261494).exp() - 0.06253275212261494

1.0019965718192345

**What is the maximum and minimum values we can work with if we're going to
Exp transform them??***

In [2]:
(f64::MAX).ln()

709.782712893384

In [9]:
(1.0/f64::MAX).ln()

-709.782712893384

In [12]:
(12.0_f64).exp()

162754.79141900392

In [8]:
(100.0_f64).exp()

2.6881171418161356e43

In [13]:
1905_f64.ln()

7.552237287560802

In [15]:
1.04f64.exp()

2.82921701435156

In [6]:
let inputs: Vec<_> = (5..126).step_by(20).collect();
inputs

[5, 25, 45, 65, 85, 105, 125]